In [8]:
import numpy as np
import matplotlib.pyplot as plt
from config import configs
import torch

In [9]:
def unnormal(data, dims = 3, dmin = None, dmax = None):
    for i in range(dims):
        data[:,i::dims] = data[:,i::dims] * (dmax[i] - dmin[i]) + dmin[i]
    return data

def normal(data):
    for i in range(data.shape[1]):
        dmax = data[:,i].max()
        dmin = data[:,i].min()
        data[:,i] = (data[:,i] - dmin) / (dmax - dmin)
    return data

def calc_psnr(img1, img2):
    img1t = torch.tensor(normal(img1))
    img2t = torch.tensor(normal(img2))
    return 10. * torch.log10(1. / torch.mean((img1t - img2t) ** 2)).item()

In [10]:
va = np.load(configs.val_path, allow_pickle = True).item()['variables']
minl = []
maxl = []
for dim in configs.dims:
    minl.append(va[dim][0])
    maxl.append(va[dim][1])
pred = np.load('result/pred.npy')
true = np.load('result/true.npy')
diff = np.abs(pred - true)
print(diff.mean())
pred = unnormal(np.load('result/pred.npy'), 2, minl, maxl)
true = unnormal(np.load('result/true.npy'), 2, minl, maxl)

0.028286846


In [11]:
pred.shape

(433, 24, 2, 64, 64)

In [12]:
mae = []
mse = []
psnr = []

for d in range(2):
    mae.append(torch.nn.functional.l1_loss(torch.tensor(pred[:,:,d::2]), torch.tensor(true[:,:,d::2])).item())
    mse.append(torch.nn.functional.mse_loss(torch.tensor(pred[:,:,d::2]), torch.tensor(true[:,:,d::2])).item())
    psnr.append(calc_psnr(pred[:,:,d::2], true[:,:,d::2]))

In [13]:
for i in range(2):
    print(mae[i])
for i in range(2):
    print(mse[i])
for i in range(2):
    print(psnr[i])

0.6810325384140015
0.7987245917320251
1.0346953868865967
1.410548448562622
20.447452068328857
22.857861518859863


In [ ]:
0.060640204697847366
0.09641994535923004
0.006348756607621908
0.014250955544412136
21.973114013671875
18.461560010910034